In [29]:
CLASSES = ''
SIZE = ""

FOLDER = ""
DATA_FOLDER = ""
FOLDS_PATH = "".format(SIZE, CLASSES)
FOLDS = 10

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
from time import ctime

import json
import os
import numpy as np
import pandas as pd
import sklearn.svm as svm

from scipy import stats
from sklearn import preprocessing
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.svm import SVC as svc 

In [31]:
created_at = ctime()
created_at = created_at.replace(' ', '-')

#Definindo funções de métrica

In [32]:
accuracy = make_scorer(precision_score)
f1 = make_scorer(f1_score, average='macro')
precision = make_scorer(precision_score, average='macro')
recall = make_scorer(recall_score, average='macro')

scorer = make_scorer(f1_score, average='macro')

In [33]:
def evaluate(clf, X, y):
  y_pred = clf.predict(X)
  acc = accuracy_score(y, y_pred)
  prec = precision_score(y, y_pred, average='macro')
  rec = recall_score(y, y_pred, average='macro')
  f1_s = f1_score(y, y_pred, average='macro')
  print(y)
  print(y_pred)
  result = [acc, prec, rec, f1_s]
  return result

In [16]:
params = {}

#Carregando dataset

In [34]:
def load_train_test(fold):
  df_train = pd.read_csv(FOLDS_PATH+"train"+fold+".csv", sep='\t')
  df_test = pd.read_csv(FOLDS_PATH+"test"+fold+".csv", sep='\t')


  cols = [str(i) for i in range(100)]

  df_train['embedding'] = df_train.loc[:,'0':'99'].values.tolist()
  df_test['embedding'] = df_test.loc[:,'0':'99'].values.tolist()

  df_train.drop(cols, axis=1, inplace=True)
  df_test.drop(cols, axis=1, inplace=True)


  train_features = df_train['embedding']
  train_labels = df_train['y']

  test_features = df_test['embedding']
  test_labels = df_test['y']


  X = []
  for t in train_features:
    X.append(np.array(t))
  train_features = np.array(X)

  X = []
  for t in test_features:
    X.append(np.array(t))
  test_features = np.array(X)

  return train_features, train_labels, test_features, test_labels

#Definindo modelo

In [24]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]    
    gammas = [0.001, 0.01, 0.1, 1, 10, 100]    
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svc(kernel='rbf'), param_grid, cv=nfolds, scoring=scorer, verbose=1)
    grid_search.fit(X, y)    
    grid_search.best_params_
    return grid_search

In [7]:
res = []
for fold in range(FOLDS):
  train_features, train_labels, test_features, test_labels = load_train_test(str(fold))
  grid_search = svc_param_selection(train_features, train_labels, 10)
  params['grid'] = grid_search.best_params_
  clf = svc(kernel='rbf', C=params['grid']['C'], gamma=params['grid']['gamma']) 
  clf.fit(train_features, train_labels)
  grid_res = evaluate(clf, test_features, test_labels)
  res.append(grid_res)

NameError: ignored

In [ ]:
res

0.6979591836734694


#Salvando Resultados

In [ ]:
n = "grid"
created_at = ctime()
created_at = created_at.replace(' ', '-')
name = "{}-{}-{}-{}-class-results.csv".format(n, SIZE, created_at, CLASSES)
with open(os.path.join(FOLDER, 'results', name), 'w') as f:
  f.write("acc, prec, rec, f1\n")
  f.write("{},{},{},{}\n".format(grid_res[0], grid_res[1], grid_res[2], grid_res[3]))  

#Random Search

In [12]:
mdl = svc(probability = True, random_state = 1)
rand_list = {"C": stats.uniform(2, 10),
             "gamma": stats.uniform(0.1, 1)}
              
rand_search = RandomizedSearchCV(mdl, param_distributions = rand_list, n_iter = 20, n_jobs = 4, cv = 3, random_state = 2017, scoring = scorer, verbose=3) 


In [13]:
res = []
train_features, train_labels, test_features, test_labels = load_train_test(str('0'))
rand_search.fit(train_features, train_labels)
print(rand_search.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:  7.3min finished


{'C': 11.307729610869862, 'gamma': 0.7495504104423961}


In [ ]:
bp = rand_search.best_params_

In [35]:
res = []
for fold in range(FOLDS):
  train_features, train_labels, test_features, test_labels = load_train_test(str(fold))
  # clf = svc(kernel='rbf', C=9.012736172332481, gamma=0.7293403609566396) 
  clf = svc(kernel='rbf', C=11.307729610869862, gamma=0.7495504104423961) 
  clf.fit(train_features, train_labels)
  rand_res = evaluate(clf, test_features, test_labels)
  res.append(rand_res)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0      0
1      0
2      0
3      2
4      1
      ..
700    2
701    2
702    0
703    2
704    1
Name: y, Length: 705, dtype: int64
[0 0 2 2 2 2 2 0 2 0 0 0 2 2 2 0 2 2 0 0 2 0 2 0 2 2 0 0 0 2 2 2 2 0 2 0 2
 0 2 0 2 2 2 0 2 2 2 2 0 2 2 2 2 0 2 0 0 0 0 0 2 0 2 2 0 0 2 2 2 0 2 2 2 2
 2 2 2 0 2 2 2 0 2 2 2 2 2 0 0 2 2 0 0 0 2 0 0 0 2 2 2 2 2 2 2 2 0 2 2 0 2
 2 0 0 2 0 2 2 2 0 0 0 0 2 2 2 0 2 2 2 2 2 0 0 2 2 0 0 0 2 2 0 2 0 2 0 0 2
 2 2 2 2 0 2 2 0 2 2 0 0 0 0 2 2 2 2 2 2 0 0 2 2 0 2 0 0 2 2 2 2 0 0 2 0 0
 0 0 0 2 0 2 0 0 2 0 2 0 2 2 2 0 2 0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 2 2 2 2 2
 0 0 2 2 0 2 0 0 2 2 0 0 0 0 0 2 2 0 2 2 2 2 0 0 0 2 0 0 0 2 2 0 2 0 0 2 2
 0 0 2 2 0 2 2 2 2 0 2 0 0 2 2 2 2 0 0 2 2 0 2 2 2 2 2 2 2 0 2 0 0 2 0 0 0
 0 0 2 0 0 2 2 0 0 0 2 0 2 0 0 0 2 2 0 0 0 2 2 0 0 0 2 2 2 2 2 0 2 2 2 2 0
 0 2 0 2 0 0 2 0 0 0 0 2 2 0 2 2 2 0 2 2 0 2 0 0 0 0 0 2 0 2 0 0 0 2 2 2 0
 0 0 0 0 2 2 0 2 0 0 0 0 2 0 0 2 2 0 0 0 0 2 0 0 0 2 0 0 2 0 2 2 0 0 2 2 2
 0 2 2 0 0 2 0 0 2 2 2 2 0 2 0 2 2 2 0 0 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0      0
1      0
2      0
3      0
4      0
      ..
700    2
701    0
702    2
703    2
704    1
Name: y, Length: 705, dtype: int64
[0 0 0 0 0 0 2 2 0 0 2 2 2 2 2 2 0 0 2 2 2 2 0 0 2 2 0 2 2 2 0 0 2 2 2 0 0
 2 0 2 2 0 0 2 0 0 0 2 0 2 2 2 2 2 0 2 0 0 2 0 2 0 0 0 0 2 0 0 2 2 2 2 2 0
 0 0 0 2 0 2 2 0 2 0 2 0 0 0 0 0 2 0 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 2 0 2 2
 2 0 0 2 2 2 2 0 0 2 2 2 2 0 0 2 0 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 0 2 2 2 2
 0 0 2 0 2 2 0 0 0 2 2 2 0 2 0 0 0 2 0 2 2 2 2 0 0 0 0 2 2 2 2 2 0 0 0 2 2
 0 2 2 2 2 2 2 2 0 2 0 2 2 2 0 2 2 0 0 0 0 0 0 2 2 2 0 0 0 0 2 0 0 0 0 2 2
 2 0 0 2 0 0 0 2 0 2 0 0 0 2 0 0 0 2 2 2 2 2 0 2 0 2 0 2 2 0 0 2 0 0 0 2 2
 2 0 2 2 0 0 2 0 0 0 0 2 2 2 0 0 2 0 2 0 2 0 0 2 2 2 0 0 0 2 2 2 2 2 2 0 2
 0 0 0 2 2 0 0 2 0 2 2 0 0 2 2 2 2 0 0 2 2 0 0 0 2 0 2 0 2 2 0 2 2 2 2 2 2
 2 2 2 2 2 0 2 0 0 0 0 2 2 0 2 2 2 0 2 0 0 0 0 2 0 0 2 0 2 2 2 2 2 0 2 0 2
 0 2 0 2 2 2 0 2 0 0 0 0 2 2 2 0 0 0 0 2 0 0 2 0 2 0 0 2 0 2 0 0 0 0 2 0 2
 0 2 2 2 0 2 0 0 0 0 2 0 0 2 2 2 0 0 0 2 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0      2
1      2
2      0
3      1
4      0
      ..
700    2
701    0
702    2
703    2
704    0
Name: y, Length: 705, dtype: int64
[0 2 0 2 0 2 2 2 2 2 2 2 2 2 0 0 0 2 2 0 2 2 0 2 2 2 2 0 0 0 0 2 0 2 2 2 2
 2 0 2 0 2 0 2 2 0 2 2 2 0 2 0 0 2 0 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 0 0
 0 2 0 2 2 0 2 0 2 2 0 2 2 0 0 0 2 0 2 2 2 0 0 2 2 2 2 2 2 2 2 0 0 2 0 2 0
 2 2 0 2 2 0 2 0 0 0 2 0 2 0 2 0 2 2 2 2 0 2 0 2 0 0 2 0 2 2 0 2 0 2 2 0 2
 2 0 2 0 2 2 0 2 2 2 0 2 0 0 0 2 2 2 0 2 0 2 0 0 0 0 0 2 2 0 2 0 0 0 2 0 2
 0 2 2 0 0 2 0 2 0 2 0 0 2 2 0 0 2 0 0 2 0 2 2 0 0 2 0 2 2 2 0 2 2 2 2 0 0
 0 2 2 2 2 0 0 2 2 0 2 0 0 2 0 0 0 0 0 0 2 0 0 2 2 2 0 0 2 2 2 2 0 0 0 2 0
 2 0 0 2 2 2 0 2 2 0 2 0 0 0 0 0 2 2 2 2 2 0 2 2 0 0 2 0 2 2 0 2 2 2 0 2 0
 0 2 2 2 0 2 2 2 0 0 0 2 2 2 2 0 2 0 2 0 0 0 2 0 2 2 0 0 0 2 2 0 0 2 0 2 0
 2 2 2 0 0 2 0 2 2 2 2 2 2 2 0 0 0 0 2 2 0 2 0 0 2 2 0 0 2 0 0 0 0 0 2 0 0
 2 0 0 0 2 2 0 0 0 0 0 2 2 2 0 0 0 2 2 2 0 0 0 2 0 0 0 2 0 2 0 0 2 0 0 0 0
 2 0 2 0 2 2 0 0 2 2 0 0 0 0 0 2 0 2 2 0 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0      0
1      0
2      2
3      0
4      0
      ..
699    0
700    2
701    1
702    2
703    1
Name: y, Length: 704, dtype: int64
[2 0 2 0 0 0 2 0 0 2 0 0 0 2 2 2 2 0 2 0 0 0 2 0 2 0 0 0 2 0 0 0 2 0 2 0 0
 2 2 2 0 2 2 0 0 2 2 0 2 0 2 0 2 0 2 2 2 2 0 0 2 2 0 2 0 0 2 2 2 2 0 2 0 2
 2 0 2 2 2 0 2 2 2 2 2 0 0 0 2 2 0 0 2 0 2 0 2 2 0 0 2 2 2 0 2 2 2 2 2 2 0
 0 0 0 2 2 0 2 2 0 2 0 2 0 2 2 2 0 0 2 0 0 2 0 0 0 0 0 2 0 2 2 0 0 2 2 2 2
 2 2 0 0 2 0 2 0 0 0 2 0 2 2 0 0 0 2 0 2 2 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 2
 0 0 2 2 2 0 2 2 0 0 0 2 2 0 0 2 2 2 2 2 0 2 2 0 0 0 2 0 2 2 0 2 2 0 0 2 0
 2 2 0 0 0 2 0 2 0 0 2 2 0 2 2 0 0 2 2 2 2 0 0 0 2 2 0 0 2 0 0 2 0 2 2 0 2
 2 0 0 0 2 0 2 2 2 0 0 0 2 0 0 2 2 2 2 2 2 0 0 0 2 2 2 2 2 2 0 0 2 0 2 2 0
 2 0 0 0 0 2 2 2 0 2 2 0 2 2 2 0 0 2 0 0 0 2 2 2 2 2 0 2 0 0 2 2 2 2 0 0 0
 0 0 2 2 0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 0 2 2 2 0 2 0 0 2 2
 2 0 0 0 0 2 0 2 2 0 2 0 2 2 2 2 0 0 0 0 2 0 0 0 2 2 0 0 2 2 2 2 0 0 0 2 0
 2 2 2 2 0 0 2 2 0 2 0 0 2 2 0 2 0 2 2 0 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
res = np.array(res)
acc = res[:,0]
prec = res[:,1]
rec = res[:,2]
f = res[:,3]

acc_avg = np.average(acc)
prec_avg = np.average(prec)
rec_avg = np.average(rec)
f1_avg = np.average(f)

acc_std = np.std(res[:,0])
prec_std = np.std(res[:,1])
rec_std = np.std(res[:,2])
f1_std = np.std(res[:,3])


print(np.std(acc))
print(np.std(prec))
print(np.std(rec))
print(np.std(f))

0.016549783775261347
0.019574410589135553
0.017420320503793576
0.01810382191434988


In [20]:
res

array([[0.71631206, 0.70220953, 0.6862203 , 0.69066465],
       [0.73475177, 0.72756242, 0.69864261, 0.70496745],
       [0.74326241, 0.7311713 , 0.71889766, 0.72315453],
       [0.77588652, 0.77451606, 0.74425034, 0.75243146],
       [0.75177305, 0.74376663, 0.72186987, 0.72810576],
       [0.73475177, 0.72444935, 0.70329568, 0.70892532],
       [0.73333333, 0.72165674, 0.70412976, 0.70922477],
       [0.74715909, 0.73603347, 0.72218639, 0.72684152],
       [0.76846591, 0.76268615, 0.74029027, 0.74701438],
       [0.74147727, 0.73557692, 0.70628077, 0.71293118]])

#Salvando Resultados

In [21]:
n = "rand"

name = "{}-{}-{}-{}-class-results.csv".format(n, SIZE, created_at, CLASSES)
with open(os.path.join(FOLDER, 'results', name), 'w') as f:
  f.write("acc_avg, prec_avg, rec_avg, f1_avg, acc_std, prec_std, rec_std, f1_std\n")
  f.write("{},{},{},{},{},{},{},{}\n".format(acc_avg, prec_avg, rec_avg, f1_avg, acc_std, prec_std, rec_std, f1_std))  

In [ ]:
params['rand'] = rand_search.best_params_

NameError: ignored

In [ ]:
with open(FOLDER+"params-{}.json".format(created_at), "w") as f:
  json.dump(params, f)